In [1]:
import importlib
import types
from eth2spec.config.config_util import prepare_config
from eth2spec.utils.ssz.ssz_impl import hash_tree_root

In [2]:
import os, sys
sys.path.insert(1, os.path.realpath(os.path.pardir) + "/notebooks/thunderdome")
import beaconrunner as br

In [3]:
import pandas as pd

In [4]:
prepare_config(".", "fast.yaml") # 4 slots per epoch
br.reload_package(br)

In [5]:
from beaconrunner.specs import SLOTS_PER_EPOCH, SECONDS_PER_SLOT
print("SLOTS_PER_EPOCH: ", SLOTS_PER_EPOCH)

SLOTS_PER_EPOCH:  4


In [6]:
import beaconrunner.simulator as simulator
import beaconrunner.network as network
from beaconrunner.validators.RANDAOValidator import RANDAOValidator

num_validators = 4
validators = [RANDAOValidator(i) for i in range(num_validators)]

# Create a genesis state
genesis_state = br.simulator.get_genesis_state(validators, seed="let's play randao")
# Validators load the state
[v.load_state(genesis_state.copy()) for v in validators]
simulator.skip_genesis_block(validators) # nothing to propose at genesis, skip to slot 1!

For simplicity let's just assume a fully connected network, meaning that all validators are connected with each other.

In [7]:
set_a = network.NetworkSet(validators=list([0,1,2,3]))
net = network.Network(validators = validators, sets = list([set_a]))

In [8]:
print("Genesis time =", validators[0].store.genesis_time, "seconds")
print("Store time =", validators[0].store.time, "seconds")
print("Current slot =", validators[0].data.slot)

Genesis time = 1578182400 seconds
Store time = 1578182412 seconds
Current slot = 1


In [9]:
def get_current_epoch_proposers():
    return get_epoch_proposers(epochs_ahead=0)

def get_next_epoch_proposers():
    return get_epoch_proposers(epochs_ahead=1)

def get_epoch_proposers(epochs_ahead=0):
    validator = net.validators[0]
    current_slot = br.specs.get_current_slot(validator.store)

    current_head_root = validator.get_head()
    current_state = validator.store.block_states[current_head_root].copy()
    current_epoch = br.specs.get_current_epoch(current_state)
    start_slot = br.specs.compute_start_slot_at_epoch(current_epoch)
    start_state = current_state.copy() if start_slot == current_state.slot else \
        validator.store.block_states[br.specs.get_block_root(current_state, current_epoch)].copy()
    
    current_epoch_proposers = []
    
    for slot in range(epochs_ahead * SLOTS_PER_EPOCH + start_slot, start_slot + (epochs_ahead+1) * SLOTS_PER_EPOCH):
        if slot < start_state.slot:
            continue
        if start_state.slot < slot:
            br.specs.process_slots(start_state, slot)
        current_epoch_proposers.append(br.specs.get_beacon_proposer_index(start_state))
    return current_epoch_proposers

In [10]:
for i in range(10):
    print("Proposer indices for epoch ", i, ": ", get_epoch_proposers(epochs_ahead=i))

Proposer indices for epoch  0 :  [3, 0, 2, 1]
Proposer indices for epoch  1 :  [3, 1, 0, 3]
Proposer indices for epoch  2 :  [0, 0, 2, 0]
Proposer indices for epoch  3 :  [1, 3, 2, 2]
Proposer indices for epoch  4 :  [0, 3, 0, 1]
Proposer indices for epoch  5 :  [0, 0, 3, 0]
Proposer indices for epoch  6 :  [1, 0, 0, 1]
Proposer indices for epoch  7 :  [3, 1, 3, 1]
Proposer indices for epoch  8 :  [3, 2, 0, 0]
Proposer indices for epoch  9 :  [0, 1, 2, 2]


We now have a schedule for who is expected to propose when. But hold on, what about the parameters `MIN_SEED_LOKOK_AHEAD=1` and `MAX_SEED_LOOKAHEAD=4`?!? Good catch! 

The `MAX_SEED_LOOKAHEAD` is actually the minimum delay on validator activations and exits; it basically means that validators strategically activating and exiting can only affect the seed 4 epochs into the future, leaving a space of 3 epochs within which proposers can mix-in unknown info to scramble the seed and hence make stake grinding via activating or exiting validators non-viable.

A random seed is used to select all the committees and proposers for an epoch. Every epoch, the beacon chain accumulates randomness from proposers via the RANDAO and stores it. The seed for the current epoch is based on the RANDAO output from the epoch `MIN_SEED_LOOKUP + 1` ago. With `MIN_SEED_LOOKAHEAD` set to one, the effect is that we can know the seed for the current epoch and the next epoch, but not beyond (since the next-but-one epoch depends on randomness from the current epoch that hasn't been accumulated yet).

Let's see if this is true, i.e. "Don't trust, but verify!"

### Simulations

Let's first create some observer functions that extract relevant data at each timestep for us

In [54]:
def current_slot(params, step, sL, s, _input):
    return ("current_slot", s["network"].validators[0].data.slot)

def current_epoch(params, step, sL, s, _input):
    return ("current_epoch", s["network"].validators[0].data.current_epoch)

# # # BROKEN NONWORKING REFACTORED VERSION
# # # TODO: refactor get_current_proposer_indices() and get_next_proposer_indices(), i.e. reuse code. Question: How to pass argument to observer function within context of cadcad?
# # # def get_epoch_proposers(validator, epochs_ahead=0):
# # #     current_slot = br.specs.get_current_slot(validator.store)
# # #     current_head_root = validator.get_head()
# # #     current_state = validator.store.block_states[current_head_root].copy()
# # #     current_epoch = br.specs.get_current_epoch(current_state)
# # #     start_slot = br.specs.compute_start_slot_at_epoch(current_epoch)
# # #     start_state = current_state.copy() if start_slot == current_state.slot else \
# # #         validator.store.block_states[br.specs.get_block_root(current_state, current_epoch)].copy()
    
# # #     epoch_proposers = []
    
# # #     for slot in range(epochs_ahead * SLOTS_PER_EPOCH + start_slot, start_slot + (epochs_ahead+1) * SLOTS_PER_EPOCH):
# # #         if slot < start_state.slot:
# # #             continue
# # #         if start_state.slot < slot:
# # #             br.specs.process_slots(start_state, slot)
# # #         epoch_proposers.append(br.specs.get_beacon_proposer_index(start_state))
# # #     return epoch_proposers

# # # def get_current_proposer_indices(params, step, sL, s, _input):
# # #     get_epoch_proposers(validator=s["network"].validators[0], epochs_ahead=0)

# # # def get_next_proposer_indices(params, step, sL, s, _input):
# # #     get_epoch_proposers(validator=s["network"].validators[0], epochs_ahead=1)


### NEW ATTEMPT
def get_current_proposer_indices(params, step, sL, s, _input):
    validator = s["network"].validators[0]
    current_slot = br.specs.get_current_slot(validator.store) #gets the slot right ...
    current_head_root = validator.get_head() # always (!) gets the same current_head_root
    print("current head root: ", current_head_root)
    current_state = validator.store.block_states[current_head_root].copy()
    current_epoch = br.specs.get_current_epoch(current_state)
    start_slot = br.specs.compute_start_slot_at_epoch(current_epoch)
    start_state = current_state.copy() if start_slot == current_state.slot else validator.store.block_states[br.specs.get_block_root(current_state, current_epoch)].copy()

    current_epoch_proposers = []
    for slot in range(start_slot, start_slot + SLOTS_PER_EPOCH):
        if slot < start_state.slot:
            continue
        if start_state.slot < slot:
            br.specs.process_slots(start_state, slot)
        current_epoch_proposers.append(br.specs.get_beacon_proposer_index(start_state))
    return ("current_epoch_proposer_indices", current_epoch_proposers)

### BROKEN BUT WORKING VERSION
# def get_current_proposer_indices(params, step, sL, s, _input):
#     validator = s["network"].validators[0]
#     current_slot = br.specs.get_current_slot(validator.store)
#     current_head_root = validator.get_head()
#     current_state = validator.store.block_states[current_head_root].copy()
#     current_epoch = br.specs.get_current_epoch(current_state)
#     start_slot = br.specs.compute_start_slot_at_epoch(current_epoch)
#     start_state = current_state.copy() if start_slot == current_state.slot else validator.store.block_states[br.specs.get_block_root(current_state, current_epoch)].copy()

#     current_epoch_proposers = []
#     for slot in range(start_slot, start_slot + SLOTS_PER_EPOCH):
#         if slot < start_state.slot:
#             continue
#         if start_state.slot < slot:
#             br.specs.process_slots(start_state, slot)
#         current_epoch_proposers.append(br.specs.get_beacon_proposer_index(start_state))
#     return ("current_epoch_proposer_indices", current_epoch_proposers)

### BROKEN BUT WORKING VERSION
# def get_next_proposer_indices(params, step, sL, s, _input):
#     validator = s["network"].validators[0]
#     current_slot = br.specs.get_current_slot(validator.store)
#     current_head_root = validator.get_head()
#     current_state = validator.store.block_states[current_head_root].copy()
#     current_epoch = br.specs.get_current_epoch(current_state)
#     start_slot = br.specs.compute_start_slot_at_epoch(current_epoch)
#     start_state = current_state.copy() if start_slot == current_state.slot else \
#         validator.store.block_states[br.specs.get_block_root(current_state, current_epoch)].copy() # get_block_root() only returns the block root of the checkpoint (!), given an epoch.

#     next_epoch_proposers = []
#     for slot in range(start_slot + SLOTS_PER_EPOCH, start_slot + 2*SLOTS_PER_EPOCH):
#         # If slot is behind start_state.slot, "jump" to slot == start_state.slot
#         if slot < start_state.slot:
#             continue
#         # If start_state.slot is behind slots we are interested in, process_slots() to slot in question. This is where we skip to next epoch... 
#         if start_state.slot < slot:
#             br.specs.process_slots(start_state, slot)
#         next_epoch_proposers.append(br.specs.get_beacon_proposer_index(start_state))
#     return ("next_epoch_proposer_indices", next_epoch_proposers)

observers = {
    "current_epoch": current_epoch,
    "current_slot": current_slot,
    "current_epoch_proposer_indices": get_current_proposer_indices,
    # "next_epoch_proposer_indices": get_next_proposer_indices,
}

In [55]:
from random import sample

def simulate_once(network_sets, num_run, num_validators, network_update_rate, scenario="honest"):
    # Generate ASAP validators
    validators = [RANDAOValidator(i) for i in range(num_validators)]
    
    # Create a genesis state
    genesis_state = br.simulator.get_genesis_state(validators, seed="let's play randao")
    
    # Validators load the state
    [v.load_state(genesis_state.copy()) for v in validators]

    br.simulator.skip_genesis_block(validators) # forward time by SECONDS_PER_SLOT

    network = br.network.Network(validators = validators, sets=network_sets)

    parameters = br.simulator.SimulationParameters({
        "num_epochs": 4,
        "num_run": num_run,
        "frequency": 1,
        "network_update_rate": network_update_rate,
        "scenario": scenario,
    })

    return br.simulator.simulate(network, parameters, observers)

In [56]:
import pandas as pd

num_validators = 4

# Create the network peers
set_a = br.network.NetworkSet(validators=list(range(num_validators)))
network_sets = list([set_a])

num_runs = 1
network_update_rate = 1

scenarios = ["honest", "skip", "slashable"]
scenario = scenarios[1]

# comment/un-comment for simulating single scenario. Specify scenario above!
df = pd.concat([simulate_once(network_sets, num_run, num_validators, network_update_rate, scenario) for num_run in range(num_runs)])

# comment/un-comment for iterating over all scenarios at once
# df = pd.concat([simulate_once(network_sets, num_run, num_validators, network_update_rate, scenarios[i]) for i in range(len(scenarios)) for num_run in range(num_runs)])

will simulate 4 epochs ( 16 slots ) at frequency 1 moves/second
total 192 simulation steps
current head root:  0xad80f24850908baecc5d6b961370c4f6b1a6b0df8093b4bc0843bf5c67beaa6e

                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (192, 3, 1, 4)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
current head root:  0xad80f24850908baecc5d6b961370c4f6b1a6b0df8093b4bc0843bf5c67beaa6e
0 proposing block for slot 1
current head root:  0xad80f24850908baecc5d6b961370c4f6b1a6b0df8093b4bc0843bf5c67beaa6e
current head root:  0xad80f24850908baecc5d6b961370c4f6b1a6b0df8093b4bc0843bf5c67beaa6e
current head root:  0xad80f24850908baecc5d6b961370c4f6b1a6b0df8093

In [58]:
df[3::3*12]

,network,current_epoch,current_slot,current_epoch_proposer_indices,simulation,subset,run,substep,timestep
3,Network(validators=[<beaconrunner.validators.R...,0,1,"[3, 0, 2, 1]",0,0,1,3,1
39,Network(validators=[<beaconrunner.validators.R...,0,2,"[3, 0, 2, 1]",0,0,1,3,13
75,Network(validators=[<beaconrunner.validators.R...,0,3,"[3, 0, 2, 1]",0,0,1,3,25
111,Network(validators=[<beaconrunner.validators.R...,1,4,"[3, 0, 2, 1]",0,0,1,3,37
147,Network(validators=[<beaconrunner.validators.R...,1,5,"[3, 0, 2, 1]",0,0,1,3,49
183,Network(validators=[<beaconrunner.validators.R...,1,6,"[3, 0, 2, 1]",0,0,1,3,61
219,Network(validators=[<beaconrunner.validators.R...,1,7,"[3, 0, 2, 1]",0,0,1,3,73
255,Network(validators=[<beaconrunner.validators.R...,2,8,"[3, 0, 2, 1]",0,0,1,3,85
291,Network(validators=[<beaconrunner.validators.R...,2,9,"[3, 0, 2, 1]",0,0,1,3,97
327,Network(validators=[<beaconrunner.validators.R...,2,10,"[3, 0, 2, 1]",0,0,1,3,109


## TODOs

- Fix get_next_proposer_indices() - this is necessary to be able to compare predicted proposers and actually realised proposers
- Manually type up scenarios (experiment A, B & C)
  - Create a slashlable proposer event
  - Punish slashable event

### Experiment A
In slot 31 let the proposer skip her duties, then forward state to slot 0 of epoch 1, record the proposer.

We are using "fast" specs (`SLOTS_PER_EPOCH = 4`). So skip block proposal for slot `3`, otherwise proceed as usual

### Experiment B
In slot 31 of epoch 0, a block was proposed for slot 31, then forward the state to slot 0 of epoch 1, record the proposer.

We are using "fast" specs (`SLOTS_PER_EPOCH = 4`). So propose a block in slot `3` as per usual and then forward state to next epoch. 

Are the proposers different between experiment A and B?

### Experiment C

The block at slot 31 of epoch 0 has a slashing event. Forward the state to slot 0 of epoch 1. Is it still the same proposer?

We are using "fast" specs (`SLOTS_PER_EPOCH = 4`). Create a slashable event for block proposer of slot `3`. Then forward state to next epoch and record proposers. 

Can we confirm the intuition that the proposer in experiment C is different than in experiment A&B?

# TODOs
- Fix observer policy function
- Implement slashing
- Why so many print statements when running simulation?!
  
